### Boiler Plate Code

In [1]:
import random

import numpy as np
from time import time
from QAP_Heuristic import QAP_Heuristic
from QAP_Tester import QAP_Hueristic_Tester
%matplotlib inline

##### Iterated Local Search

We closely follow the paper: ILS for QAP (Sultze '05) to implement the below method

In [3]:
class IteratedLocalSearch(QAP_Heuristic):
    def __init__(self, w, d) -> None:
        super().__init__(w, d)


    def __str__():
        return "ils"
    
    # Subroutines
    def generate_initial_solution(self):
        return np.random.permutation(self.n)
    
    
    def local_search(self, perm: np.array):
        n = len(perm)
        curr_best = np.copy(perm)  # create a deep copy
        
        for i in range(1, n-2):
            for j in range(i+1, n):
                
                new_perm = np.copy(curr_best)
                new_perm[i:j] = curr_best[j-1:i-1:-1]
                
                if self.cost(new_perm) < self.cost(curr_best): 
                    curr_best = new_perm        
        
        return curr_best
    
    
    def acceptance_criterion(self, perm1: np.array, perm2: np.array):
        return perm1 if self.cost(perm1) < self.cost(perm2) else perm2
    
    @staticmethod
    def peturbation(perm: np.array, k: int):
        new_perm = perm[:]
        # of course, this inexact method can result in pertubations of size < k, but these occur with sufficiently small probability 
        for _ in range(k): 
            i = random.randint(0, len(perm)-1)
            j = random.randint(0, len(perm)-1)
            new_perm[i], new_perm[j] = new_perm[j], new_perm[i]   
        
        return new_perm
    

    # ILS implementation
    def solve(self, n_iters: int):
        s0 = self.generate_initial_solution()
        s  = self.local_search(s0)
        
        curr_best = s
        cost_history = [self.cost(s)]
        perm_history = [s]
        n = len(s0)

        for _ in range(n_iters):
            
            if time() > self.MAX_CPU_TIME: break
            
            s1 = self.peturbation(s, k=n//2)
            s2 = self.local_search(s1)
            curr_best = self.acceptance_criterion(curr_best, s2)
            
            cost_history.append(self.cost(curr_best))
            perm_history.append(curr_best)

        return curr_best

### Automated Testing
code for automating the opening, closing and testing of a hueristic on every instance in QAPLib

In [4]:
instance_path = "../QAPInstances/"
solution_path = "../QAPSolns/"
write_to_path = "../results/"

In [5]:
tester = QAP_Hueristic_Tester(heuristic=IteratedLocalSearch, 
                              instance_path=instance_path,
                              soln_path=solution_path,
                              write_to_path=write_to_path)

In [9]:
tester.test_hueristic(n_iters=10_000, n_trials=5, tai_only=True)